У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [272]:
%pip install -U imbalanced-learn
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from imblearn.over_sampling import SMOTENC, SMOTEN
from imblearn.combine import SMOTETomek
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_curve, auc, f1_score, confusion_matrix, roc_auc_score, r2_score, mean_squared_error
from imblearn.datasets import fetch_datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier


onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
scaler = MinMaxScaler()
imputer_frequent = SimpleImputer(strategy='most_frequent')
imputer_mean = SimpleImputer(strategy='mean')

df = pd.read_csv('customer_segmentation_train.csv')
df_test = pd.read_csv('customer_segmentation_test.csv')
target_col = 'Segmentation'
df.head(5)
df.nunique()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ID                 8068
Gender                2
Ever_Married          2
Age                  67
Graduated             2
Profession            9
Work_Experience      15
Spending_Score        3
Family_Size           9
Var_1                 7
Segmentation          4
dtype: int64

In [273]:
#df.drop(columns=['ID'], inplace=True)
numeric_cols = df.select_dtypes(include='number').columns
categorical_cols = df.drop(columns=[target_col]).select_dtypes(include='object').columns

df_converted = df.copy()

imputer_frequent.fit(df_converted[categorical_cols])
df_converted[categorical_cols] = imputer_frequent.transform(df_converted[categorical_cols])
df_test[categorical_cols] = imputer_frequent.transform(df_test[categorical_cols])
imputer_mean.fit(df_converted[numeric_cols])
df_converted[numeric_cols] = imputer_mean.transform(df_converted[numeric_cols])
df_test[numeric_cols] = imputer_mean.transform(df_test[numeric_cols])

scaler.fit(df_converted[numeric_cols])
df_converted[numeric_cols] = scaler.transform(df_converted[numeric_cols])
df_test[numeric_cols] = scaler.transform(df_test[numeric_cols])

onehot.fit(df_converted[categorical_cols])
encoded_cols = list(onehot.get_feature_names_out(categorical_cols))

df_test[encoded_cols] = onehot.transform(df_test[categorical_cols])
df_converted[encoded_cols] = onehot.transform(df_converted[categorical_cols])
df_converted.drop(columns=categorical_cols, inplace=True)
df_converted.columns

X_train, X_val, y_train, y_val = train_test_split(df_converted.drop(columns=[target_col]), df[target_col], test_size=0.2, random_state=12)


In [274]:
df_converted.head()

,ID,Age,Work_Experience,Family_Size,Segmentation,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,...,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,0.425601,0.056338,0.071429,0.375,D,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.407140,0.281690,0.188690,0.250,A,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.815503,0.690141,0.071429,0.000,B,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.306161,0.690141,0.000000,0.125,B,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.410031,0.309859,0.188690,0.625,A,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [275]:
cat_feature_indx = [X_train.columns.get_loc(col) for col in encoded_cols]
smotenc = SMOTENC(categorical_features=cat_feature_indx, random_state=42)
X_train_resampled_smotenc, y_train_resampled_smotenc = smotenc.fit_resample(X_train, y_train)
print(y_train.value_counts())
print(y_train_resampled_smotenc.value_counts())


Segmentation
D    1826
A    1573
C    1565
B    1490
Name: count, dtype: int64
Segmentation
C    1826
B    1826
D    1826
A    1826
Name: count, dtype: int64


In [276]:
X_train_resampled_tomek, y_train_resampled_tomek = SMOTETomek(random_state=42).fit_resample(X_train, y_train)
print(y_train.value_counts())
print(y_train_resampled_tomek.value_counts())

Segmentation
D    1826
A    1573
C    1565
B    1490
Name: count, dtype: int64
Segmentation
D    1437
C    1434
A    1357
B    1354
Name: count, dtype: int64


In [277]:
X_train_resampled_smoten, y_train_resampled_smoten = SMOTEN(random_state=42).fit_resample(X_train, y_train)
print(y_train.value_counts())
print(y_train_resampled_smoten.value_counts())

Segmentation
D    1826
A    1573
C    1565
B    1490
Name: count, dtype: int64
Segmentation
C    1826
B    1826
D    1826
A    1826
Name: count, dtype: int64


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [278]:
def createOvRModelAndRate( X_train, y_train, X_val, y_val):
    log_reg = LogisticRegression(solver='liblinear')
    ovr_model = OneVsRestClassifier(log_reg)
    ovr_model.fit(X_train, y_train)
    ovr_predictions = ovr_model.predict(X_val)
    y_pred = ovr_predictions
    print("OneVsRest\n",classification_report(y_val, y_pred))

def createOvOModelAndRate( X_train, y_train, X_val, y_val):
    log_reg = LogisticRegression(solver='liblinear')
    ovo_model = OneVsOneClassifier(log_reg)
    ovo_model.fit(X_train, y_train)
    ovo_predictions = ovo_model.predict(X_val)
    y_pred = ovo_predictions
    print("OneVsOne\n",classification_report(y_val, y_pred))
X_val_enc = X_val.copy()
X_val_enc = X_val_enc[X_train_resampled_smotenc.columns] 
createOvRModelAndRate(X_train_resampled_smotenc, y_train_resampled_smotenc, X_val_enc, y_val)
createOvOModelAndRate(X_train_resampled_smotenc, y_train_resampled_smotenc, X_val_enc, y_val)


OneVsRest
               precision    recall  f1-score   support

           A       0.44      0.52      0.48       399
           B       0.38      0.24      0.29       368
           C       0.53      0.63      0.58       405
           D       0.70      0.69      0.69       442

    accuracy                           0.53      1614
   macro avg       0.51      0.52      0.51      1614
weighted avg       0.52      0.53      0.52      1614

OneVsOne
               precision    recall  f1-score   support

           A       0.43      0.47      0.45       399
           B       0.37      0.32      0.34       368
           C       0.55      0.58      0.56       405
           D       0.70      0.70      0.70       442

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.51      1614
weighted avg       0.52      0.52      0.52      1614



In [279]:
createOvRModelAndRate(X_train_resampled_tomek, y_train_resampled_tomek, X_val, y_val)
createOvOModelAndRate(X_train_resampled_tomek, y_train_resampled_tomek, X_val, y_val)

OneVsRest
               precision    recall  f1-score   support

           A       0.42      0.51      0.46       399
           B       0.38      0.23      0.28       368
           C       0.54      0.66      0.59       405
           D       0.70      0.66      0.68       442

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.50      1614
weighted avg       0.52      0.52      0.51      1614

OneVsOne
               precision    recall  f1-score   support

           A       0.45      0.47      0.46       399
           B       0.39      0.33      0.36       368
           C       0.55      0.61      0.58       405
           D       0.70      0.69      0.69       442

    accuracy                           0.53      1614
   macro avg       0.52      0.52      0.52      1614
weighted avg       0.53      0.53      0.53      1614



In [280]:
createOvRModelAndRate(X_train_resampled_smoten, y_train_resampled_smoten, X_val, y_val)
createOvOModelAndRate(X_train_resampled_smoten, y_train_resampled_smoten, X_val, y_val)

OneVsRest
               precision    recall  f1-score   support

           A       0.45      0.52      0.48       399
           B       0.39      0.24      0.30       368
           C       0.53      0.64      0.58       405
           D       0.69      0.69      0.69       442

    accuracy                           0.53      1614
   macro avg       0.52      0.52      0.51      1614
weighted avg       0.52      0.53      0.52      1614

OneVsOne
               precision    recall  f1-score   support

           A       0.44      0.47      0.45       399
           B       0.37      0.30      0.33       368
           C       0.55      0.60      0.58       405
           D       0.69      0.70      0.69       442

    accuracy                           0.53      1614
   macro avg       0.51      0.52      0.51      1614
weighted avg       0.52      0.53      0.52      1614



In [281]:
createOvRModelAndRate(X_train, y_train, X_val, y_val)
createOvOModelAndRate(X_train, y_train, X_val, y_val)

OneVsRest
               precision    recall  f1-score   support

           A       0.44      0.51      0.47       399
           B       0.40      0.20      0.26       368
           C       0.53      0.65      0.58       405
           D       0.69      0.72      0.70       442

    accuracy                           0.53      1614
   macro avg       0.51      0.52      0.51      1614
weighted avg       0.52      0.53      0.52      1614

OneVsOne
               precision    recall  f1-score   support

           A       0.44      0.47      0.45       399
           B       0.39      0.28      0.32       368
           C       0.55      0.61      0.58       405
           D       0.69      0.73      0.71       442

    accuracy                           0.53      1614
   macro avg       0.52      0.52      0.52      1614
weighted avg       0.52      0.53      0.53      1614



Я порівняла стратегії і моделі по Ф1 метриці, стратегії мають краще результати із OneVsOne. Сильної різниці немає, я умаю через те що сам набір даних не сильно великий і моделі не можуть добре натренуватися. Бачимо що всюди найкращий результат показує клас D, так як ми ним заповнювали нульові колонки то це логічно. 